# LCEL(LangChain Expression Language)

一个简单的示例

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("给我讲一个关于 {topic}的笑话")
model = ChatOpenAI(model="gpt-4")
output_parser = StrOutputParser()

# 表达式
chain = prompt | model | output_parser

chain.invoke({"topic": "冰激凌"})


In [ ]:
prompt_value = prompt.invoke({"topic": "刺猬"})
prompt_value.to_messages()
# [HumanMessage(content='给我讲一个关于 刺猬的笑话')]

message = model.invoke(prompt_value)
message
# AIMessage(content='一只兔子和一只刺猬赛跑，刺猬以微弱的优势赢了...', response_metadata={'finish_reason': 'stop', 'logprobs': None})

# 使用 LLM 的区别
llm = OpenAI(model="gpt-3.5-turbo-instruct")
llm.invoke(prompt_value)
# '\n\nRobot: 为什么刺猬的生日派对总是很失败？因为他们总是把蜡烛都弄灭了！'

## RAG Search Exampl

比较复杂的例子

- 建立向量数据
- 使用RAG增强

In [ ]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# 创建向量数据库
vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
# 使用
chain.invoke("where did harrison work?")
# 'Harrison worked at Kensho.'

In [ ]:
# 自定义 template
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke({"question": "where did harrison work", "language": "chinese"})

到 6.11